# **Data Extraction from ENTSOE**

---
# methods that return Pandas Series
client.query_day_ahead_prices(country_code, start=start, end=end)
client.query_net_position(country_code, start=start, end=end, dayahead=True)
client.query_crossborder_flows(country_code_from, country_code_to, start=start, end=end)
client.query_scheduled_exchanges(country_code_from, country_code_to, start=start, end=end, dayahead=False)
client.query_net_transfer_capacity_dayahead(country_code_from, country_code_to, start=start, end=end)
client.query_net_transfer_capacity_weekahead(country_code_from, country_code_to, start=start, end=end)
client.query_net_transfer_capacity_monthahead(country_code_from, country_code_to, start=start, end=end)
client.query_net_transfer_capacity_yearahead(country_code_from, country_code_to, start=start, end=end)
client.query_intraday_offered_capacity(country_code_from, country_code_to, start=start, end=end, implicit=True)
client.query_offered_capacity(country_code_from, country_code_to, contract_marketagreement_type, start=start, end=end, implicit=True)
client.query_aggregate_water_reservoirs_and_hydro_storage(country_code, start=start, end=end)

# methods that return Pandas DataFrames
client.query_load(country_code, start=start, end=end)
client.query_load_forecast(country_code, start=start, end=end)
client.query_load_and_forecast(country_code, start=start, end=end)
client.query_generation_forecast(country_code, start=start, end=end)
client.query_wind_and_solar_forecast(country_code, start=start, end=end, psr_type=None)
client.query_intraday_wind_and_solar_forecast(country_code, start=start, end=end, psr_type=None)
client.query_generation(country_code, start=start, end=end, psr_type=None)
client.query_generation_per_plant(country_code, start=start, end=end, psr_type=None, include_eic=False)
client.query_installed_generation_capacity(country_code, start=start, end=end, psr_type=None)
client.query_installed_generation_capacity_per_unit(country_code, start=start, end=end, psr_type=None)
client.query_imbalance_prices(country_code, start=start, end=end, psr_type=None)
client.query_contracted_reserve_prices(country_code, type_marketagreement_type, start=start, end=end, psr_type=None)
client.query_contracted_reserve_amount(country_code, type_marketagreement_type, start=start, end=end, psr_type=None)
client.query_unavailability_of_generation_units(country_code, start=start, end=end, docstatus=None, periodstartupdate=None, periodendupdate=None)
client.query_unavailability_of_production_units(country_code, start, end, docstatus=None, periodstartupdate=None, periodendupdate=None)
client.query_unavailability_transmission(country_code_from, country_code_to, start=start, end=end, docstatus=None, periodstartupdate=None, periodendupdate=None)
client.query_withdrawn_unavailability_of_generation_units(country_code, start, end)
client.query_physical_crossborder_allborders(country_code, start, end, export=True)
client.query_generation_import(country_code, start, end)
client.query_procured_balancing_capacity(country_code, process_type, start=start, end=end, type_marketagreement_type=None)

Mappings : https://github.com/EnergieID/entsoe-py/blob/master/entsoe/mappings.py


In [3]:
!pip install entsoe-py
#!pip install dwdweather2
!pip install wetterdienst

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00


In [4]:
from entsoe import EntsoeRawClient,EntsoePandasClient
import pandas as pd
client_pd = EntsoePandasClient(api_key='9a9df453-dc5d-40a7-8400-1c2f6bd53d9b')
start_ds1 = pd.Timestamp('20230101', tz='Europe/Berlin')
start_ds2 = pd.Timestamp('20200101', tz='Europe/Berlin')
end = pd.Timestamp('20240101', tz='Europe/Berlin')
country_code = "DE"
country_bidding_zone = "DE_LU"

**Day-ahead prices**

In [5]:
day_ahead_prices_ds2 = client_pd.query_day_ahead_prices(country_bidding_zone, start=start_ds2, end=end)
day_ahead_prices_ds1 = client_pd.query_day_ahead_prices(country_bidding_zone, start=start_ds1, end=end)

**Load**

In [ ]:
df_load_ds2 = client_pd.query_load(country_code, start=start_ds2, end=end)
df_load_ds1 = client_pd.query_load(country_code, start=start_ds1, end=end)


**Generation**

In [6]:
day_generation_ds2 = client_pd.query_generation(country_bidding_zone, start=start_ds2, end=end)
day_generation_ds1 = client_pd.query_generation(country_bidding_zone, start=start_ds1, end=end)

# **Weather Data**

---

https://github.com/earthobservations/wetterdienst/blob/main/examples/notebooks/wetterdienst_notebook.ipynb

**Cloud Cover**

In [ ]:
from wetterdienst.provider.dwd.observation import (
    DwdObservationDataset,
    DwdObservationParameter,
    DwdObservationPeriod,
    DwdObservationRequest,
    DwdObservationResolution)

In [ ]:
import polars as pl
_ = pl.Config.set_tbl_hide_dataframe_shape(True)
from wetterdienst import Settings
from wetterdienst.provider.dwd.observation import DwdObservationRequest
settings = Settings( # default
     ts_shape="long",  # tidy data
     ts_humanize=True,  # humanized parameters
     ts_si_units=True  # convert values to SI units
 )
request_cs = DwdObservationRequest(
    parameter="climate_summary",
    resolution="daily",
    start_date="2020-01-01",  # if not given timezone defaulted to UTC
    end_date="2024-10-01",  # if not given timezone defaulted to UTC
    settings=settings
 )#.filter_by_station_id(station_id=(1048, 4411))

In [ ]:
request_cloud= DwdObservationRequest(
    parameter=DwdObservationDataset.CLOUDINESS,
    resolution=DwdObservationResolution.HOURLY,
    start_date="2020-01-01",  # if not given timezone defaulted to UTC
    end_date="2020-12-31",
).all()
request_cloud_2020 = request_cloud.values.all()

**Sun Availability**

In [ ]:
from wetterdienst.provider.dwd.observation import (
    DwdObservationDataset,
    DwdObservationParameter,
    DwdObservationPeriod,
    DwdObservationRequest,
    DwdObservationResolution)
print(DwdObservationRequest.discover(resolution=DwdObservationResolution.HOURLY))
print(DwdObservationRequest.discover(resolution=DwdObservationResolution.DAILY))
print(DwdObservationRequest.discover())
#Extracted year by year due to size
request = DwdObservationRequest(
    parameter=DwdObservationDataset.SOLAR,
    resolution=DwdObservationResolution.HOURLY,
    start_date="2020-01-01",  # if not given timezone defaulted to UTC
    end_date="2020-12-31",
).all()
request_solar_2020 = request.values.all()
request_solar_2020 = request_solar_2020.df.to_pandas()
sunshine_duration_data = request_solar_2020[request_solar_2020["parameter"] == "sunshine_duration"]


**Humidity**

In [ ]:
request = DwdObservationRequest(
    parameter=DwdObservationDataset.TEMPERATURE_AIR,
    resolution=DwdObservationResolution.HOURLY,
    start_date="2021-01-01",  # if not given timezone defaulted to UTC
    end_date="2021-12-31",
).all()
request_temp_2021 = request.values.all()
request_temp_2021 = request_temp_2021.df.to_pandas()
request_temp_2021 = request_temp_2021[request_temp_2021['parameter']=='humidity']
request_humidity_2021_df = pd.merge(weather_stations, request_temp_2021, on='station_id', how='inner')
request_humidity_2021_df = request_humidity_2021_df[['state', 'date','value']]
request_humidity_2021_df.rename(columns={'value': 'humidity'}, inplace=True)

In [ ]:
parameters = [
    DwdObservationParameter.DAILY.TEMPERATURE_AIR_MEAN_2M,
    DwdObservationParameter.DAILY.TEMPERATURE_AIR_MAX_2M,
    DwdObservationParameter.DAILY.TEMPERATURE_AIR_MIN_2M,
    DwdObservationParameter.DAILY.PRECIPITATION_HEIGHT,
]
values_par = (
    DwdObservationRequest(
        parameter=parameters,
        resolution=DwdObservationResolution.DAILY,
        period=DwdObservationPeriod.HISTORICAL,
    )
    .filter_by_station_id(station_id=(1048,))
    .values.all()
)
values_par.df.drop_nulls().head()

In [ ]:
weather_stations = pd.read_csv("/content/drive/MyDrive/Master Thesis/Script/Datasets/stations.csv", dtype={'station_id': str}, parse_dates=True)
weather_stations.drop(columns=["Unnamed: 0"],inplace=True)
len(weather_stations.station_id.unique())

There are 1257 weather stations, selected stations : 'Baden-Württemberg', 'Nordrhein-Westfalen', 'Niedersachsen',
       'Schleswig-Holstein', 'Bayern', 'Hessen', 'Brandenburg',
       'Thüringen', 'Mecklenburg-Vorpommern', 'Sachsen',
       'Rheinland-Pfalz', 'Sachsen-Anhalt', 'Berlin', 'Saarland',
       'Bremen', 'Hamburg'

In [ ]:
request_wind_2024['wind_direction'] = request_wind_2024.loc[request_wind_2024["parameter"] == "wind_direction"]['value'] #3881123
request_wind_2024['wind_speed'] = request_wind_2024.loc[request_wind_2024["parameter"]  == "wind_speed"]['value']